# Fine-tuning InceptionV3 on custom dataset

## Imports

Please refer to the [README.md](README.md) for proper installation of the dependencies listed here.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np

import os
import sys
import glob
import random
from IPython.display import clear_output, Image, display
import PIL.Image

from keras.applications.inception_v3 import InceptionV3, preprocess_input

from keras.models import Model, load_model
from keras.layers import Dense, GlobalAveragePooling2D, AveragePooling2D, Dropout, Flatten, Conv2D, Activation
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


## Settings

- `train_dir`: Directory that holds the training samples generated by [wga.ipynb](wga.ipynb)
- `train_dir`: Directory that holds the validation samples enerated by [wga.ipynb](wga.ipynb)
- `classes`: Number of classes in the training set.
- `batch_size`: Batch size, default 32.
- `epochs`: How many epochs to train for.
- `finetune`: Are we fine-tuning or training from scratch?
- `weights`: If fine-tuning, which weights are we using to initialize?
- `size`: Input image size for InceptionV3
- `freeze`: If fine-tuning, up to which layer are we freezing the weights (164: up to and including mixed5, 133: up to and including mixed4, 101: up to and including mixed3)
- `cpath`: Save path for model files

In [ ]:
train_dir = 'wga-portrait-landscape-stilllife/train'
val_dir = 'wga-portrait-landscape-stilllife/val'
classes = 3
batch_size = 32
epochs = 10
finetune = True
weights = 'imagenet'
size = 299 # V3
freeze = 164 # V3, up to and including mixed5
cpath = 'portrait-landscape-stilllife-v3-{epoch:02d}.hdf5'

## Helper functions

In [ ]:
def add_new_last_layer_v3(base_model, classes):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(classes, activation='softmax', name='Predictions')(x)
    model = Model(input=base_model.input, output=predictions)
    return model

def setup_to_finetune(model):
    for layer in model.layers[:freeze]:
        layer.trainable = False
    for layer in model.layers[freeze:]:
        layer.trainable = True
    model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
    
def setup_to_train(model):
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

## Define Keras data generators

In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(size, size),
    batch_size=batch_size,
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(size, size),
    batch_size=batch_size,
)

## Build and train the model

In [ ]:
# We need to define the input shape, see 
# https://stackoverflow.com/questions/49043955/shape-of-input-to-flatten-is-not-fully-defined-got-none-none-64
base_model = InceptionV3(weights=weights, input_shape=(size, size, 3), include_top=False)
layer_dict = dict([(layer.name, n) for n, layer in enumerate(base_model.layers)])
model = add_new_last_layer_v3(base_model, classes)
if finetune: setup_to_finetune(model)
else: setup_to_train(model)
_ = model.fit_generator(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight='auto',
    callbacks=[ModelCheckpoint(cpath)])